In [1]:
"""
Celii adapted code for pinky 100 run 2 for direct connectivity

"""

'\nCelii adapted code for pinky 100 run 2 for direct connectivity\n\n'

In [2]:
import pycircstat as pycs

ModuleNotFoundError: No module named 'pycircstat'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools 
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import datajoint as dj
from collections import defaultdict
import pycircstat as pycs
from funconnect import morphology, connectomics, ta3, nda
from tqdm import tqdm
import scipy.stats as stats


In [ ]:
ta3p100 = dj.create_virtual_module('microns_ta3p100', 'microns_ta3p100')
ndap100 = dj.create_virtual_module('microns_ndap100', 'microns_ndap100')
radtune = dj.create_virtual_module('goldfunc_radtune', 'goldfunc_radtune')
spattune = dj.create_virtual_module('goldfunc_spattune', 'goldfunc_spattune')
fc = dj.create_virtual_module('fc', 'goldfunc_fc')


# Direct Connection

## Orientation / direction tuning 

In [ ]:
dj.ERD(ta3p100) +1 -1

In [ ]:
# Exclude synapses, segments and restrict by functional confidence 

synapse = (ta3p100.Synapse - ta3p100.SynapseExcludeFinal) & ta3p100.CurrentSegmentation
segment = (ta3p100.Segment - ta3p100.SegmentExclude) & ta3p100.CurrentSegmentation
soma = (ta3p100.AllenSoma - ta3p100.SegmentExclude) & ta3p100.CurrentSegmentation

In [ ]:
# number of direct connections (synapses) for each functional soma pair (both presyn and postsyn are somas with functional traces)

fun_fun_syn = synapse & soma.proj(presyn = 'segment_id') & soma.proj(postsyn = 'segment_id') 
n_syn = dj.U('presyn', 'postsyn').aggr(fun_fun_syn, n_syn = 'count(*)') # soma-soma pairs with at least one synapse

all_fun_fun = soma.proj(presyn = 'segment_id') * soma.proj(postsyn = 'segment_id') & 'presyn != postsyn'
rel = (dj.U('presyn', 'postsyn') & all_fun_fun).proj('presyn', 'postsyn', n_syn='0')
total_n_syn = rel.aggr(dj.U('n_add') * n_syn.proj(n_add='n_syn'), n_syn_total='n_syn + ifnull(n_add, 0)', keep_all_rows=True)# fill out all zeros

#reciprocal_pairs = (n_syn * n_syn.proj(postsyn='presyn', presyn='postsyn', n_syn_2='n_syn')).proj(n_syn = 'n_syn + n_syn_2')



In [ ]:
total_n_syn

## Should count undirected soma-soma pairs, i.e. A->B and B->A synpases summed together for connection between A and B


In [ ]:
# combine a-b and b-a connections into one number, and segment_id1 < segment_id2
attrs = ['presyn', 'postsyn', 'n_syn_total']
a = (total_n_syn & 'presyn < postsyn').proj('presyn', 'postsyn', n_syn1 = 'n_syn_total')
b = (total_n_syn & 'presyn > postsyn').proj(n_syn2 = 'n_syn_total', postsyn = 'presyn', presyn = 'postsyn')
undirected_n_syn = (a * b).proj(n_syn_total = 'n_syn1 + n_syn2', segment_id1 = 'presyn', segment_id2 = 'postsyn')


In [ ]:
# functional metrics for each functional soma pair (restricting by radtune fitting significance)

tuned = 'confidence > 0.5'
sig_units = radtune.BestVonFit.Unit & 'von_p_value <= 0.001' & tuned
sig_unit_pairs = (sig_units.proj(segment_id1 = 'segment_id') * 
                 sig_units.proj(segment_id2 = 'segment_id')) & 'segment_id1 < segment_id2'

radconn_df = pd.DataFrame(((radtune.BestVonCorr & sig_unit_pairs) * undirected_n_syn).fetch())

'''
# dataframe for directed connections, i.e. a-b different from b-a

attrs = ['von_hash1', 'von_hash2', 'segmentation', 'segment_id1', 'segment_id2', 'diff_pref_ori', 'diff_pref_dir', 'diff_sharp', 'diff_osi', 'diff_dsi', 'von_corr', 'von_corr_pvalue', 'diff_v_adv', 'diff_v_pvalue']

radconn_pairs1 = (radtune.BestVonCorr & sig_unit_pairs).proj(*attrs, presyn = 'segment_id1', postsyn = 'segment_id2') * total_n_syn
radconn_pairs2 = (radtune.BestVonCorr & sig_unit_pairs).proj(*attrs, presyn = 'segment_id2', postsyn = 'segment_id1') * total_n_syn

radconn_df1 = pd.DataFrame(radconn_pairs1.fetch())
radconn_df2 = pd.DataFrame(radconn_pairs2.fetch())
radconn_df = pd.concat([radconn_df1, radconn_df2], ignore_index=True)
'''


In [ ]:
sum(radconn_df['n_syn_total'].values)

In [ ]:
radconn_df['n_syn_total'].values

In [ ]:
# set up bins for difference in preferred direction
rad2deg = 180/np.pi
dir_edges = np.linspace(0, np.pi, 5)
de = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in dir_edges]])
dir_labels = list(zip(de[:-1], de[1:]))
dir_centers = np.round((dir_edges[1:] + dir_edges[:-1])/2 * rad2deg, decimals=2) 

In [ ]:
# set up bins for difference in prefered orientation 
ori_edges = np.linspace(0, np.pi /2 ,5)
oe = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in ori_edges]])
ori_labels = list(zip(oe[:-1], oe[1:]))
ori_centers = np.round((ori_edges[1:] + ori_edges[:-1])/2 * rad2deg, decimals=2) 

In [ ]:
# set up bins for difference in tuning sharpness
sharp_edges = np.linspace(min(np.abs(radconn_df['diff_sharp'])), max(np.abs(radconn_df['diff_sharp'])), 6)
se = list(['{:.1f}'.format(ee) for ee in sharp_edges])
sharp_labels = list(zip(se[:-1], se[1:]))
sharp_centers = np.hstack((np.nan, np.round((sharp_edges[1:] + sharp_edges[:-1])/2, decimals=2), np.nan))

In [ ]:
# bin diff_pref_ori, diff_pref_dir, diff_sharp
radconn_df['bin_diff_pref_ori'] = ori_centers[(np.digitize(np.abs(radconn_df['diff_pref_ori']), ori_edges)) - 1]
radconn_df['bin_diff_pref_dir'] = dir_centers[(np.digitize(np.abs(radconn_df['diff_pref_dir']), dir_edges)) - 1]
radconn_df['bin_diff_sharp'] = sharp_centers[(np.digitize(np.abs(radconn_df['diff_sharp']), sharp_edges))]

In [ ]:
# convert n_syn_total into binary values
radconn_df['n_syn_binary'] = 1 * (radconn_df['n_syn_total'] > 0).values
radconn_df_part = radconn_df[radconn_df['n_syn_total'] > 0]

In [ ]:
# for difference in preferred orientation 

with sns.axes_style('ticks'):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
sns.pointplot('bin_diff_pref_ori', 'n_syn_binary', data = radconn_df, ax=ax[0]) # probability of direct connection
sns.pointplot('bin_diff_pref_ori', 'n_syn_total', data = radconn_df, ax=ax[1])  # average number of connections for all possible functional pairs
sns.pointplot('bin_diff_pref_ori', 'n_syn_total', data = radconn_df_part, ax=ax[2]) # average number of connections for pairs with at least one direct connection

ax[0].set_title('Probability of direct connection')
ax[0].set_xticklabels(['{}°-{}°'.format(*a) for a in ori_labels])
ax[0].set_xlabel(r'$\Delta \theta$')
ax[0].set_ylabel('$P_{conn}$')

ax[1].set_title('Avg conn for all possible functional pairs')
ax[1].set_xticklabels(['{}°-{}°'.format(*a) for a in ori_labels])
ax[1].set_xlabel(r'$\Delta \theta$')
ax[1].set_ylabel('$<Direct connections>$')

ax[2].set_title('Avg conn for pairs with >=1 conn')
ax[2].set_xticklabels(['{}°-{}°'.format(*a) for a in ori_labels])
ax[2].set_xlabel(r'$\Delta \theta$')
ax[2].set_ylabel('$<Direct connections>$')

sns.despine(trim=True)
fig.tight_layout()



In [ ]:
# for difference in preferred direction 

with sns.axes_style('ticks'):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
sns.pointplot('bin_diff_pref_dir', 'n_syn_binary', data = radconn_df, ax=ax[0]) # probability of direct connection
sns.pointplot('bin_diff_pref_dir', 'n_syn_total', data = radconn_df, ax=ax[1])  # average number of connections for all possible functional pairs
sns.pointplot('bin_diff_pref_dir', 'n_syn_total', data = radconn_df_part, ax=ax[2]) # average number of connections for pairs with at least one direct connection

ax[0].set_title('Probability of direct connection')
ax[0].set_xticklabels(['{}°-{}°'.format(*a) for a in dir_labels])
ax[0].set_xlabel(r'$\Delta \theta$')
ax[0].set_ylabel('$P_{conn}$')

ax[1].set_title('Avg conn for all possible functional pairs')
ax[1].set_xticklabels(['{}°-{}°'.format(*a) for a in dir_labels])
ax[1].set_xlabel(r'$\Delta \theta$')
ax[1].set_ylabel('$<Direct connections>$')

ax[2].set_title('Avg conn for pairs with >=1 conn')
ax[2].set_xticklabels(['{}°-{}°'.format(*a) for a in dir_labels])
ax[2].set_xlabel(r'$\Delta \theta$')
ax[2].set_ylabel('$<Direct connections>$')

sns.despine(trim=True)
fig.tight_layout()


In [ ]:
# for difference in tuning sharpness

with sns.axes_style('ticks'):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
sns.pointplot('bin_diff_sharp', 'n_syn_binary', data = radconn_df, ax=ax[0]) # probability of direct connection
sns.pointplot('bin_diff_sharp', 'n_syn_total', data = radconn_df, ax=ax[1])  # average number of connections for all possible functional pairs
sns.pointplot('bin_diff_sharp', 'n_syn_total', data = radconn_df_part, ax=ax[2]) # average number of connections for pairs with at least one direct connection

ax[0].set_title('Probability of direct connection')
ax[0].set_xticklabels(['{}-{}'.format(*a) for a in sharp_labels])
ax[0].set_xlabel(r'$\Delta$ tuning sharpness')
ax[0].set_ylabel('$P_{conn}$')

ax[1].set_title('Avg conn for all possible functional pairs')
ax[1].set_xticklabels(['{}-{}'.format(*a) for a in sharp_labels])
ax[1].set_xlabel(r'$\Delta$ tuning sharpness')
ax[1].set_ylabel('$<Direct connections>$')

ax[2].set_title('Avg conn for pairs with >=1 conn')
ax[2].set_xticklabels(['{}-{}'.format(*a) for a in sharp_labels])
ax[2].set_xlabel(r'$\Delta$ tuning sharpness')
ax[2].set_ylabel('$<Direct connections>$')

sns.despine(trim=True)
fig.tight_layout()


## Receptive field

In [ ]:
# functional metrics for each functional soma pair (restricting by spattune fitting significance)


sig_units = spattune.BestSTA.Loc & 'sta_snr > 1.5' & segment
sig_unit_pairs = (sig_units.proj(segment_id1 = 'segment_id') * 
                  sig_units.proj(segment_id2 = 'segment_id')) & 'segment_id1 < segment_id2'

spatconn_df = pd.DataFrame(((spattune.BestSTACorr & sig_unit_pairs) * undirected_n_syn).fetch())

'''
# dataframe for directed connections, i.e. a-b different from b-a

attrs = ['sta_hash1', 'sta_hash2', 'apert_radius', 'segmentation', 'segment_id1', 'segment_id2', 'center_dist', 'apert_overlap', 'full_corr_r2', 'full_corr_pvalue', 
         'union_corr_r2', 'union_corr_pvalue', 'intersect_corr_r2', 'intersect_corr_pvalue', 'align_corr_r2', 'align_corr_pvalue', 'align_overlap', 'diff_snr', 'diff_snr_pvalue']

spatconn_pairs1 = (spattune.BestSTACorr & sig_unit_pairs).proj(*attrs, presyn = 'segment_id1', postsyn = 'segment_id2') * total_n_syn
spatconn_pairs2 = (spattune.BestSTACorr & sig_unit_pairs).proj(*attrs, presyn = 'segment_id2', postsyn = 'segment_id1') * total_n_syn

spatconn_df1 = pd.DataFrame(spatconn_pairs1.fetch())
spatconn_df2 = pd.DataFrame(spatconn_pairs2.fetch())
spatconn_df = pd.concat([spatconn_df1, spatconn_df2])
'''


In [ ]:
spatconn_df

In [ ]:
# set up bins for `union_corr_r2`
union_edges = np.linspace(min(spatconn_df['union_corr_r2']), max(spatconn_df['union_corr_r2']), 6)
ue = list(['{:.2f}'.format(ee) for ee in union_edges])
union_labels = list(zip(ue[:-1], ue[1:]))
union_centers = np.hstack((np.nan, np.round((union_edges[1:] + union_edges[:-1])/2, decimals=2), np.nan))

In [ ]:
# bin `union_corr_r2`
spatconn_df['bin_union_corr_r2'] = union_centers[(np.digitize(spatconn_df['union_corr_r2'], union_edges))]


In [ ]:
# convert n_syn_total into binary values
spatconn_df['n_syn_binary'] = 1 * (spatconn_df['n_syn_total'] > 0).values
spatconn_df_part = spatconn_df[spatconn_df['n_syn_total'] > 0]

In [ ]:
# for difference in preferred orientation 

with sns.axes_style('ticks'):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
sns.pointplot('bin_union_corr_r2', 'n_syn_binary', data = spatconn_df, ax=ax[0]) # probability of direct connection
sns.pointplot('bin_union_corr_r2', 'n_syn_total', data = spatconn_df, ax=ax[1])  # average number of connections for all possible functional pairs
sns.pointplot('bin_union_corr_r2', 'n_syn_total', data = spatconn_df_part, ax=ax[2]) # average number of connections for pairs with at least one direct connection

ax[0].set_title('Probability of direct connection')
ax[0].set_xticklabels(['[{},{}]'.format(*a) for a in union_labels], rotation=30)
ax[0].set_xlabel('Union sta correlation r2')
ax[0].set_ylabel('$P_{conn}$')

ax[1].set_title('Avg conn for all possible functional pairs')
ax[1].set_xticklabels(['[{},{}]'.format(*a) for a in union_labels], rotation=30)
ax[1].set_xlabel('Union sta correlation r2')
ax[1].set_ylabel('$<Direct connections>$')

ax[2].set_title('Avg conn for pairs with >=1 conn')
ax[2].set_xticklabels(['[{},{}]'.format(*a) for a in union_labels], rotation=30)
ax[2].set_xlabel('Union sta correlation r2')
ax[2].set_ylabel('$<Direct connections>$')

sns.despine(trim=True)
fig.tight_layout()



# Shared Input

## Orientation / Direction

In [ ]:
sig_units = radtune.BestVonFit.Unit & 'von_p_value <= 0.001' & segment
sig_unit_pairs = (sig_units.proj(segment_id1 = 'segment_id') * 
                 sig_units.proj(segment_id2 = 'segment_id')) & 'segment_id1 < segment_id2'

attrs = ['n_syn_a', 'n_syn_b', 'n_syn_union', 'n_syn_shared', 'n_seg_a', 'n_seg_b', 'n_seg_union', 'n_seg_shared']
radconn_df = pd.DataFrame(((radtune.BestVonCorr & sig_unit_pairs) * fc.SharedInput.proj(*attrs, segment_id1 = 'segment_id', segment_id2 = 'segment_b')).fetch())

In [ ]:
radconn_df

In [ ]:
# bin diff_pref_ori, diff_pref_dir, diff_sharp
radconn_df['bin_diff_pref_ori'] = ori_centers[(np.digitize(np.abs(radconn_df['diff_pref_ori']), ori_edges)) - 1]
radconn_df['bin_diff_pref_dir'] = dir_centers[(np.digitize(np.abs(radconn_df['diff_pref_dir']), dir_edges)) - 1]
radconn_df['bin_diff_sharp'] = sharp_centers[(np.digitize(np.abs(radconn_df['diff_sharp']), sharp_edges))]

In [ ]:
# for difference in preferred orientation 
radconn_df['percent_shared'] = radconn_df['n_seg_shared'] / radconn_df['n_seg_union']

with sns.axes_style('ticks'):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
sns.pointplot('bin_diff_pref_ori', 'n_seg_shared', data = radconn_df, ax=ax[0]) 
sns.pointplot('bin_diff_pref_ori', 'n_seg_union', data = radconn_df, ax=ax[1])  
sns.pointplot('bin_diff_pref_ori', 'percent_shared', data = radconn_df, ax=ax[2]) 
ax[0].set_title('Shared inputs')
ax[0].set_xticklabels(['{}°-{}°'.format(*a) for a in ori_labels])
ax[0].set_xlabel(r'$\Delta \theta$')
ax[0].set_ylabel('$<Shared Input Segments>$')

ax[1].set_title('Union inputs')
ax[1].set_xticklabels(['{}°-{}°'.format(*a) for a in ori_labels])
ax[1].set_xlabel(r'$\Delta \theta$')
ax[1].set_ylabel('$<Union Input Segments>$')

ax[2].set_title('Percent of shared inputs')
ax[2].set_xticklabels(['{}°-{}°'.format(*a) for a in ori_labels])
ax[2].set_xlabel(r'$\Delta \theta$')
ax[2].set_ylabel('$<Shared / Union>$')

sns.despine(trim=True)
fig.tight_layout()



In [ ]:
# for difference in preferred direction of motion 

with sns.axes_style('ticks'):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
sns.pointplot('bin_diff_pref_dir', 'n_seg_shared', data = radconn_df, ax=ax[0]) 
sns.pointplot('bin_diff_pref_dir', 'n_seg_union', data = radconn_df, ax=ax[1])  
sns.pointplot('bin_diff_pref_dir', 'percent_shared', data = radconn_df, ax=ax[2]) 
ax[0].set_title('Shared inputs')
ax[0].set_xticklabels(['{}°-{}°'.format(*a) for a in dir_labels])
ax[0].set_xlabel(r'$\Delta \theta$')
ax[0].set_ylabel('$<Shared Input Segments>$')

ax[1].set_title('Union inputs')
ax[1].set_xticklabels(['{}°-{}°'.format(*a) for a in dir_labels])
ax[1].set_xlabel(r'$\Delta \theta$')
ax[1].set_ylabel('$<Union Input Segments>$')

ax[2].set_title('Percent of shared inputs')
ax[2].set_xticklabels(['{}°-{}°'.format(*a) for a in dir_labels])
ax[2].set_xlabel(r'$\Delta \theta$')
ax[2].set_ylabel('$<Shared / Union>$')

sns.despine(trim=True)
fig.tight_layout()



In [ ]:
# for difference in tuning sharpness

with sns.axes_style('ticks'):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
sns.pointplot('bin_diff_sharp', 'n_seg_shared', data = radconn_df, ax=ax[0]) 
sns.pointplot('bin_diff_sharp', 'n_seg_union', data = radconn_df, ax=ax[1])  
sns.pointplot('bin_diff_sharp', 'percent_shared', data = radconn_df, ax=ax[2]) 
ax[0].set_title('Shared inputs')
ax[0].set_xticklabels(['[{},{}]'.format(*a) for a in sharp_labels])
ax[0].set_xlabel(r'$\Delta$ tuning sharpness')
ax[0].set_ylabel('$<Shared Input Segments>$')

ax[1].set_title('Union inputs')
ax[1].set_xticklabels(['[{},{}]'.format(*a) for a in sharp_labels])
ax[1].set_xlabel(r'$\Delta$ tuning sharpness')
ax[1].set_ylabel('$<Union Input Segments>$')

ax[2].set_title('Percent of shared inputs')
ax[2].set_xticklabels(['[{},{}]'.format(*a) for a in sharp_labels])
ax[2].set_xlabel(r'$\Delta$ tuning sharpness')
ax[2].set_ylabel('$<Shared / Union>$')

sns.despine(trim=True)
fig.tight_layout()



## Receptive field

In [ ]:
# functional metrics for each functional soma pair (restricting by spattune fitting significance)

sig_units = spattune.BestSTA.Loc & 'sta_snr > 1.5' & segment
sig_unit_pairs = (sig_units.proj(segment_id1 = 'segment_id') * 
                  sig_units.proj(segment_id2 = 'segment_id')) & 'segment_id1 < segment_id2'

attrs = ['n_syn_a', 'n_syn_b', 'n_syn_union', 'n_syn_shared', 'n_seg_a', 'n_seg_b', 'n_seg_union', 'n_seg_shared']
spatconn_df = pd.DataFrame(((spattune.BestSTACorr & sig_unit_pairs) * fc.SharedInput.proj(*attrs, segment_id1 = 'segment_id', segment_id2 = 'segment_b')).fetch())


In [ ]:
# set up bins for `union_corr_r2`
union_edges = np.linspace(min(spatconn_df['union_corr_r2']), max(spatconn_df['union_corr_r2']), 6)
ue = list(['{:.2f}'.format(ee) for ee in union_edges])
union_labels = list(zip(ue[:-1], ue[1:]))
union_centers = np.hstack((np.nan, np.round((union_edges[1:] + union_edges[:-1])/2, decimals=2), np.nan))

In [ ]:
# bin `union_corr_r2`
spatconn_df['bin_union_corr_r2'] = union_centers[(np.digitize(spatconn_df['union_corr_r2'], union_edges))]


In [ ]:
# for union_corr_r2 
spatconn_df['percent_shared'] = spatconn_df['n_seg_shared'] / spatconn_df['n_seg_union']
 
with sns.axes_style('ticks'):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
sns.pointplot('bin_union_corr_r2', 'n_seg_shared', data = spatconn_df, ax=ax[0]) 
sns.pointplot('bin_union_corr_r2', 'n_seg_union', data = spatconn_df, ax=ax[1])  
sns.pointplot('bin_union_corr_r2', 'percent_shared', data = spatconn_df, ax=ax[2]) 
ax[0].set_title('Shared inputs')
ax[0].set_xticklabels(['[{},{}]'.format(*a) for a in union_labels], rotation=30)
ax[0].set_xlabel('Union sta correlation r2')
ax[0].set_ylabel('$<Shared Input Segments>$')

ax[1].set_title('Union inputs')
ax[1].set_xticklabels(['[{},{}]'.format(*a) for a in union_labels], rotation=30)
ax[1].set_xlabel('Union sta correlation r2')
ax[1].set_ylabel('$<Union Input Segments>$')

ax[2].set_title('Percent of shared inputs')
ax[2].set_xticklabels(['[{},{}]'.format(*a) for a in union_labels], rotation=30)
ax[2].set_xlabel('Union sta correlation r2')
ax[2].set_ylabel('$<Shared / Union>$')

sns.despine(trim=True)
fig.tight_layout()

